In [12]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.constants as C
import pandas as pd

In [13]:

def match(P, Po, k):
    a = (k-1)/k
    return np.sqrt((2/(k-1))*((Po/P)**a - 1))

def RatArea(M1, M2, k):
    a=(k+1)/(2*(k-1))
    b=(k-1)/2
    return (M2/M1)*(((1+b*(M1**2))/(1+b*(M2**2)))**a)

def A1A2(Po,P,k):
    a = (k-1)/k
    b = (((k+1)/2)**(1/(k-1)))*((P/Po)**(1/k))
    c = np.sqrt((k+1)/(k-1)*(1-(P/Po)**a))
    return b*c

def Cf(Pe, P0, k):
    a=(k+1)/(k-1)
    return np.sqrt((2*k**2/(k-1)*(2/(k+1))**a)*(1-(Pe/P0)**((k-1)/k)))

def thrust(Cf,At,P0):
    return At*P0*Cf*6894.757*10**(-6)

def Isp(k, Pe, Po, To, M):
    R=C.R
    a=(k-1)/k
    Isp=(1/C.g)*np.sqrt(((2*To*(R/(M*a))))*(1-(Pe/Po)**a))
    return Isp

def diametro(A):
    return np.sqrt((4*A)/C.pi)

def altura(m, mp, F, Isp, angulo):
    ay=(F/m)*np.sin(angulo)-C.g
    
    # Fase 1
    t1=(Isp)*mp*C.g/F
    h1=1/2*ay*t1**2
    if h1>3000 :
        print("La fase 1 pasa de 3000 metros")
    
    V_y1=ay*t1
    
    # Fase 2
    t2=V_y1/C.g

    h=(ay*t1**2/2)*(1+ay/C.g)
    return h

def masa_propelente(F, Isp, m, h,angulo):
    ay=(F/m)*np.sin(angulo)-C.g
    mp=(F/Isp)*np.sqrt(2*h/(C.g*ay)*(1/(C.g+ay)))
    return mp

def ex_vel(R, To, k):
    a = (k+1)/(k-1)
    c = np.sqrt(R*To/(k*(2/(k+1))**a))
    return c


In [14]:
a=RatArea(match(14.7,1000,1.15),1  , 1.15)
b=A1A2(1000,14.7, 1.15 ) # AtAe
print(a, match(14.7,1000,1.15), 1/b)

9.834508456093598 3.1283348359674954 9.834508456093598


# Al Can V2 propelente

In [15]:
P0 = 1000 #psi
Pe = 14.69 #psi=presion atmosferica
k = 1.136231 #
M = 0.03686465 # kg/mol
To = 1855.11 #1940.74 #K
theta = np.pi/2
r = 275/6 # mm/s
rho_T = 1.8805 #densidad teorica
rho_e = 1.39 # g/cm^3 densidad medida
mp=0.350 # peso del propelente utilizado en la prueba

print(rho_e/rho_T)

0.7391651156607284


In [16]:
print(1/A1A2(P0,Pe, k))

10.138900641635141


In [17]:
m=50 # masa del cohete sin propelente
h=3000 # metros
theta=np.pi/2 #angulo

Mt=1
Me=match(Pe,P0,k)
Dc=101.6 #mm
Ac=C.pi*(Dc/2)**2 #mm**2

## Tobera

In [18]:
AtAc=1/4 #RatArea(Mt,Mc,k)
At=Ac*AtAc
AeAt = RatArea(Me, Mt, k)
Ae=At*AeAt
#AeAc=RatArea(Me,Mc,k)

#Ae2=Ac*AeAc
#De2=diametro(Ae2)

De=diametro(Ae)
Dt=diametro(At)
Cf=Cf(Pe,P0,k)
F=thrust(Cf,At,P0)
Isp=Isp(k,Pe,P0,To,M) #159 s

# Condiciones de la tobera
T_t=2*To/(k+1)
P_t=P0*(2/(k+1))**(k/(k-1))

In [19]:
#Calcular la presión de camara
Ab = np.pi*(38**2)/4
Kn = Ab/(np.pi*(3/2)**2)
Po=Kn*rho_e*r*ex_vel(C.R/M, To , k)/10**(6)*145.038 # psi
print(Po, Kn, ex_vel(C.R/M, To , k), Dc, Dt)

1508.1573214222421 160.44444444444443 1017.2879889033136 101.6 50.8


In [20]:
# Datos de la tobera
print(Dc,Dt,De,Me,Mt,Cf,F, Dc/Dt, AeAt, P0, T_t, P_t) # Dt,Dc=mm match,Cf=sin dimendiones, F=N

101.6 50.8 161.75553329586378 3.1097281583697245 1 1.6365306719233723 22869.697471943633 2.0 10.138900641635137 1000 1736.8065532238788 577.1790057768961


Para poder obtener la masa total incluyendo el propelente se puede se puede resolver con una serie geometrica de la forma
$$
\frac{M}{m}=\Sigma_{n=0}^\infty (MR)^n=\frac{1}{1-MR} ,    |MR|<1
$$
Para poder obtener un MR razonable se necesita hacer un promedio de la funcion de mp/m=MR de la forma
$$
 MR=\langle \frac{mp}{m} \rangle=\frac{1}{I_{sp}}\sqrt{\frac{2hF}{gsen\theta}} \frac{1}{b-a}\int_a^b\frac{dm}{\sqrt{Fsen\theta-mg}} 
$$


In [50]:
# Integral de mp/m
from scipy import integrate
h=3000
a, b = 40, 60
c=(1/Isp)*np.sqrt(2*h*F/(np.sin(theta)*C.g))
K= lambda m: c*(1/np.sqrt(F*np.sin(theta)-C.g*m))   

MR=1/(b-a)*(integrate.quadrature(K, a, b)[0]) # relacion de masa del propelente y el cohete

i = 0
m_p = [0]
M_tot = [m]
error=1
while error > 10**-6:
    
    m_p.append(M_tot[i]*K(M_tot[i]))
    M_tot.append(m+m_p[i+1])
    error= M_tot[i+1]-M_tot[i]
    #print(error, M_tot[-1])
    i+=1

Mtot=m*(1/(1-MR)) # masa total
h1=altura(Mtot ,Mtot-m, F, Isp, np.pi/2) #altura alcanzada
h2 = altura(M_tot[-1] ,m_p[-1], F, Isp, np.pi/2) # Altura con metodo iterativo
t=Isp*(Mtot-m)*C.g/F # tiempo de empuje

V_e=m_p[-1]*1000/rho_e # cm^3
V_t=m_p[-1]*1000/rho_T # cm^3
L=V_e*1000/Ac # mm
# print(MR, K(50), Mtot, h1)
print(t, F, M_tot[-1], m_p[-1], h2, V, L, Isp) # mm

0.6287122336391169 22869.697471943633 58.65587439679075 8.655874396790752 2999.999971651874 6227.247767475362 768.1019158438787 169.76470007795893


## Volumen propelente
Para obetner un buen flujo de los gases de la camara con una relación $L/D=6$
$$
\frac{A_p}{A_t}=\frac{\pi D^2(1-V_1)}{4A_t}
$$
Donde D es el diametro del propelente y V1 es la relación de propelente y volumen de la camara
$$
V_1=\frac{V_p}{V_a}
$$
Donde $V_p$ es el volumen del propelente y $V_a$ es el volumen valido de la camara. Despejando para $V_a$
$$
V_a=\frac{V_p}{1-\frac{A_p}{A_t}A_t\frac{4}{\pi D^2}}
$$

In [57]:
# Longitud de la camara 

ApAt=2
Va1 = (V_e)/(1-ApAt*At*4/(np.pi*Dc**2)) # cm^3
L_e = Va1*10**3/Ac # mm

Va2 = (V_t)/(1-ApAt*At*4/(np.pi*Dc**2)) # cm^3
L_t = Va2*10**3/Ac # mm

print(V_e/(Va1), L_e, L_t, V_t/Va2 )

0.5 1536.2038316877574 1135.5082829279354 0.5


## Datos obtenidos
Aquí se muestra en una tabla todos los datos importantes que se calcularon para la misión relacionados con motor